In [ ]:
import os
import pandas as pd
import itertools
from datetime import datetime
import plotly.graph_objects as go
from ipywidgets import widgets

class Vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)() # retain local pointer to value
        return value 

In [2]:
wells = [f"{i[1]}{i[0]}" for i in itertools.product(range(1, 13), ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])]

In [3]:
def read_raw_table(table_path):
    r = pd.read_excel(table_path, skiprows=28, index_col=0) 
    df = r.loc[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], :]
    df.index.name = None
    t = r.loc["End Time:", 1]
    t = datetime.strptime(t, '%d/%m/%Y %H:%M:%S')
    return df, t

def convert_plate_tables_to_dict(plate):
    main_dict = Vividict()
    df_dict = {}  # New dictionary to store the loaded dataframes
    
    for table_file in os.listdir(f"OD_readings/{plate}"):
        table_path = f"OD_readings/{plate}/{table_file}"
        df, t = read_raw_table(table_path)
        t = t.strftime('%Y-%m-%d %H:%M:%S')
        df_dict[t] = df  # Store the dataframe for later use
        
    for well in wells:
        ts = list(df_dict.keys())
        vals = [df.loc[well[0], int(well[1:])] for df in df_dict.values()]
        
        tmp_df = pd.DataFrame([ts, vals]).T.sort_values(0)
        main_dict[well]["x"] = tmp_df[0].tolist()
        main_dict[well]["y"] = tmp_df[1].tolist()
        
    return main_dict

In [4]:
def add_trace(go_fig, plate):
    d = convert_plate_tables_to_dict(plate)
    for well in wells:
        rec = d[well]
        ts, vals = rec["x"], rec["y"]
        fig.add_trace(
            go.Scatter(x=ts, y=vals, name=well)
        )
    return go_fig

In [5]:
layout = go.Layout(title={"text": "OD600"})

plate_w = widgets.Dropdown(
    description='Plate:   ',
    options=os.listdir("OD_readings/")
)

fig = go.FigureWidget()
fig = add_trace(fig, plate_w.value)
fig.update_layout(title_text=f'{plate_w.value}', title_x=0.5)
fig.layout.xaxis.title = 'Absorbance at 600 nm'
fig.layout.yaxis.title = 'Date time'

fig.update_layout(
    height=800
)

def response(change):
    fig.update_layout(title_text=f'Loading', title_x=0.5)
    old, new = change["old"], change["new"]
    plate = new
    fig.update_layout(title_text=f"Loading ... {plate}", title_x=0.5)
    nd = convert_plate_tables_to_dict(plate)
    
    with fig.batch_update():
        for idx, well in enumerate(wells):
            fig.data[idx].x = nd[well]["x"]
            fig.data[idx].y = nd[well]["y"]
            fig.data[idx].name = well
        fig.update_layout(title_text=f'{plate}', title_x=0.5)

    
plate_w.observe(response, names="value")

widgets.VBox([plate_w, fig])

In [9]:
print("::: Configurations :::")
%load_ext watermark
%watermark -v -m -p pandas,plotly,openpyxl,ipywidgets,voila
print()
%watermark -u -n -t -z

::: Configurations :::
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.12.0

pandas    : 1.5.1
plotly    : 5.11.0
openpyxl  : 3.0.10
ipywidgets: 7.7.2
voila     : 0.3.7

Compiler    : GCC 10.3.0
OS          : Linux
Release     : 5.15.0-76-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit


Last updated: Tue Jul 11 2023 16:32:22+08

